<a href="https://colab.research.google.com/github/gabelev/pinns/blob/main/gabe_PINNS_05052024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Authorize me
from google.colab import auth
auth.authenticate_user()

In [2]:
## Check Dedalus isn't already installed
## Installation takes about 1-2 mins

try:
    import dedalus.public as de
    print("Dedalus already installed")
except:
    print("Dedalus not installed yet. Let's do it.")

# Step 1: Install FFTW
!apt-get install libfftw3-dev
!apt-get install libfftw3-mpi-dev

# Step 2: Set paths for Dedalus installation
import os
os.environ['MPI_INCLUDE_PATH'] = "/usr/lib/x86_64-linux-gnu/openmpi/include"
os.environ['MPI_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
os.environ['FFTW_INCLUDE_PATH'] = "/usr/include"
os.environ['FFTW_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"

# Step 3: Install Dedalus using pip
!pip3 install dedalus

# Step 4: Check Dedalus is importable
print()
print()
try:
    import dedalus.public as de
    print("Dedalus successfully installed :)")
except:
    print("Error installing Dedalus :(")
    raise

Dedalus not installed yet. Let's do it.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfftw3-bin libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
Suggested packages:
  libfftw3-doc
The following NEW packages will be installed:
  libfftw3-bin libfftw3-dev libfftw3-double3 libfftw3-long3 libfftw3-quad3 libfftw3-single3
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,654 kB of archives.
After this operation, 24.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-double3 amd64 3.3.8-2ubuntu8 [770 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-long3 amd64 3.3.8-2ubuntu8 [335 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-quad3 amd64 3.3.8-2ubuntu8 [614 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfftw3-single3 amd64 3.3.8-2

2024-05-06 01:48:13,470 dedalus 0/1 WARNING :: Threading has not been disabled. This may massively degrade Dedalus performance.


2024-05-06 01:48:13,472 dedalus 0/1 WARNING :: We strongly suggest setting the "OMP_NUM_THREADS" environment variable to "1".


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


2024-05-06 01:48:13,565 numexpr.utils 0/1 INFO :: NumExpr defaulting to 8 threads.


DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


Dedalus successfully installed :)


In [3]:
"""
Dedalus script simulating the 1D Korteweg-de Vries / Burgers equation.
This script demonstrates solving a 1D initial value problem and produces
a space-time plot of the solution. It should take just a few seconds to
run (serial only).

We use a Fourier basis to solve the IVP:
    dt(u) + u*dx(u) = a*dx(dx(u)) + b*dx(dx(dx(u)))

To run and plot:
    $ python3 kdv_burgers.py
"""

import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
# import logging
# logger = logging.getLogger(__name__)

# geo_start = 0
# geo_end = 1
# time_start = 0
# time_end = 1
def num_solver(
        geo_start,
        geo_end,
        time_start,
        time_end,
    ):

    # Parameters
    # TODO set PINNS from 0,
    Lx = geo_end


    # TODO align with PINNS
    # Compare values of the solution on these points from Deadalus
    # Calculate error with the PINNS test data
    num_basis_modes = 256 # num_basis modes 2048

    # viscosity term
    a = 1e-2 / np.pi
    # set b = 0 is Burgers b = 2e-4, This is k in PINNs
    b = 0
    # padding with zeros (ratio)
    dealias = 3/2

    # same as PINNS stop time
    stop_sim_time = time_end

    # TODO: what is this?
    timestepper = d3.SBDF2
    # runs 500 * 10 = 5k
    # unit of time is 500 steps
    #
    timestep = 4*(1e-4)
    dtype = np.float64

    # Bases
    # TODO 1D?
    coordinates = d3.Coordinate('x')

    # cart_coordinates = d3.CartesianCoordinates('x')
    # cart_dist = d3.Distributor(cart_coordinates, dtype=dtype)
    # cart_basis = d3.Ca(
    #     cart_dist,
    #     size=1,
    #     bounds=(0, Lx)
    # )

    dist = d3.Distributor(coordinates, dtype=dtype)

    # TODO: Nx? Num points? Num F modes
    # bounds
    xbasis = d3.RealFourier(
        coordinates,
        size=num_basis_modes, # number of modes for the basis
        bounds=(0, Lx),
        dealias=dealias,
    )

    # Fields
    u = dist.Field(name='u', bases=xbasis)

    # Substitutions
    dx = lambda A: d3.Differentiate(A, coordinates)

    # Problem
    # Initial value problem
    problem = d3.IVP([u], namespace=locals())

    problem.add_equation("dt(u) - a*dx(dx(u)) - b*dx(dx(dx(u))) = - u*dx(u)")

    # Initial conditions
    x = dist.local_grid(xbasis)
    # n = 20
    # u['g'] = np.log(1 + np.cosh(n)**2/np.cosh(n*(x-0.2*Lx))**2) / (2*n)
    # u['g'] = -np.sin(8 * np.pi * x), # -sin(8*pi*x)
    # Be aware this needs to be adjusted to match PINNs
    u['g'] = -np.sin(np.pi * x)
    # u['g'] = initial_condition_fn

    # Solver
    solver = problem.build_solver(timestepper)
    solver.stop_sim_time = stop_sim_time

    # Main loop
    u.change_scales(1)
    u_list = [np.copy(u['g'])]
    t_list = [solver.sim_time]
    while solver.proceed:
        solver.step(timestep)
        # if solver.iteration % 100 == 0:
        #     logger.info('Iteration=%i, Time=%e, dt=%e' %(solver.iteration, solver.sim_time, timestep))
        if solver.iteration % 25 == 0:
            u.change_scales(1)
            u_list.append(np.copy(u['g']))
            t_list.append(solver.sim_time)

    return x, u, u_list, t_list


def fit_solver(x, u, u_list, t_list):
    u_list_np = np.array(u_list)
    u_list_np = u_list_np[:len(u_list_np)-1]
    t_list_np = np.array(t_list)
    t_list_np = t_list_np[:len(t_list_np)-1]
    xx, tt = np.meshgrid(x, t_list_np)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = u_list_np.flatten()[:, None]
    return X, y


In [4]:
!pip install deepxde
!pip install numpy==1.19.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 33.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [5]:
import deepxde as dde
from deepxde.backend import tf

No backend selected.
Finding available backend...


Level 1:tensorflow:Registering Relayout (<function _relayout_gradient at 0x781cf4b59000>) in gradient.
Level 1:tensorflow:Registering RelayoutLike (<function _relayout_grad_gradient at 0x781cf49d1630>) in gradient.
Level 1:tensorflow:Registering CopyToMesh (<function _copy_to_mesh_gradient at 0x781cf4b591b0>) in gradient.
Level 1:tensorflow:Registering CopyToMeshGrad (<function _copy_to_mesh_grad_gradient at 0x781cf49d1750>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxArgs (<function _FakeQuantWithMinMaxArgsGradient at 0x781cf49308b0>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxVars (<function _FakeQuantWithMinMaxVarsGradient at 0x781cf47adea0>) in gradient.
Level 1:tensorflow:Registering FakeQuantWithMinMaxVarsPerChannel (<function _FakeQuantWithMinMaxVarsPerChannelGradient at 0x781cf47ae050>) in gradient.
Level 1:tensorflow:Registering QuantizeAndDequantizeV4 (<function _QuantizeAndDequantizeV4Grad at 0x781cf4930af0>) in gradient.
Level 1:tensor

Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
2024-05-06 01:59:28,873 tensorflow 0/1 WARNING :: From /usr/local/lib/python3.10/dist-packages/tensorflow/python/compat/v2_compat.py:108: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


Level 1:tensorflow:Disabling resource variables
Level 1:tensorflow:Disabling tensor equality
Level 1:tensorflow:Disabling control flow v2
Level 1:tensorflow:Enabling v2 tensorshape
Enable just-in-time compilation with XLA.



In [6]:
dde.backend.set_default_backend("tensorflow")
#tf.enable_eager_execution()
print(tf.__version__)
print(tf.executing_eagerly())

Setting the default backend to "tensorflow". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
2.15.0
False


In [7]:
import deepxde as dde
import numpy as np
from os.path import exists



# EVAL_DATA = "deepxde/examples/dataset/Burgers.npz"


# def precheck():
#     if not exists(EVAL_DATA):
#         raise FileNotFoundError("Evaluation data not loaded into notebook")

# def gen_testdata(eval_data):
#     data = np.load(eval_data)
#     t, x, exact = data["t"], data["x"], data["usol"].T
#     xx, tt = np.meshgrid(x, t)
#     X = np.vstack((np.ravel(xx), np.ravel(tt))).T
#     y = exact.flatten()[:, None]
#     return X, y


def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, i=0, j=0)
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    dy_xxxx = dde.grad.hessian(dy_xx, x, i=0, j=0)
    viscosity = 0.01 # TODO match viscostity with Deadalus
    k = 0
    return dy_t + y * dy_x + (k * dy_xxxx) + viscosity / np.pi * dy_xx # TODO add nudged terms.



def run_train_eval(
        train_test_contidions: list,
        iters=50_000,
        original_eval=False,
        overlap=True,
    ):

    """
    Run train and evaluation

    Ags:
    - train_test_conditions: a list of dictionaries to train and evaluate
      our experiments. [
        {
            "train_time": [0.0, 0.99] # the time start/end to train the model, float
            "train_geo": [-1,1] # initial geo interval for train
            "test_time": [0.5,0.75] # the time start to test the model, float
            "test_geo": [-1, 1], # initial geo interval for test
        },
        {
            ...
        }
      ]
    - iterations: int, number of epochs to train the model

    Returns:
    - result: a list of dictionaries with the training results

    Raises:
    - FileNotFoundError if test data not on the box
    """

    # dummy check
    # precheck()



    # Set the time domain to be less than the test data

    # timedomain = dde.geometry.TimeDomain(0.0, 0.99)
    # TODO:
    # - run on different subsets of time where t!=0, ie 0.25:0.75
    # - regular intervals of certain length
    # - random intervals of time (how to randomly choose disjoint intervals)
    #   - sparse and small
    # - outcome: corelations of length and how sparsely we are in the domin with the error
    # - investigate other metrics?
    # - compare errors directly over the training intervals vs outside the intervals
    # - perform the same for space
    # - break up space for fixed time

    result = []

    i = 1

    #iterate over different time training
    for condition in train_test_contidions:
        print(f"iteration: {i}")

        # pull all the conditions
        train_time_start = condition.get("train_time_start")
        train_time_end = condition.get("train_time_end")
        train_geo_start = condition.get("train_geo_start")
        train_geo_end = condition.get("train_geo_end")
        test_time_start = condition.get("test_time_start")
        test_time_end = condition.get("test_time_end")
        test_geo_start = condition.get("test_geo_start")
        test_geo_end = condition.get("test_geo_end")

        geom = dde.geometry.Interval(train_geo_start, train_geo_end)
        timedomain = dde.geometry.TimeDomain(train_time_start, train_time_end)
        geomtime = dde.geometry.GeometryXTime(geom, timedomain)

        # TODO: is this correct?
        # icbc = initial condition Dirichlet boundry condition
        bc = dde.icbc.DirichletBC(
            geomtime,
            lambda x: 0,
            lambda _,
            on_boundary:
            on_boundary
        )

        ic = dde.icbc.IC(
            geomtime,
            lambda x: -np.sin(np.pi * x[:, 0:1]),
            #lambda x: -np.sin(8 * np.pi * x[:, 0:1]), # -sin(8*pi*x)
            lambda _,
            on_initial: on_initial
        )

        # TODO any changes around domain and boundries?
        # TODO: do we need the data to be at least at this resolution?
        data = dde.data.TimePDE(
            geomtime, # TODO how is it generating data for training
            pde,
            [bc, ic],
            num_domain=2540,
            num_boundary=80,
            num_initial=160,
            # num_test=100,
        )

        net = dde.nn.FNN([2] + [20] * 3 + [1], "tanh", "Glorot normal")
        model = dde.Model(data, net)

        model.compile("adam", lr=1e-3)
        model.train(iterations=iters, display_every=10000)
        model.compile("L-BFGS")
        losshistory, train_state = model.train()

        # test against full timedomain in test data from numerical solver?

        if original_eval:
            pass
            # X, y_true = gen_testdata(EVAL_DATA)

        else:
            x_d, u_d, u_list, t_list = num_solver(
                test_geo_start,
                test_geo_end,
                test_time_start,
                test_time_end
                )
            X, y_true = fit_solver(x_d, u_d, u_list, t_list)


        y_pred = model.predict(X)

        # TODO, not sure we want to use this?
        f = model.predict(X, operator=pde)
        mean_residual = np.mean(np.absolute(f))



        rel_error = dde.metrics.l2_relative_error(y_true, y_pred)
        metric = condition
        metric.update({
            # "conditions": condition,
            "mean_residual": mean_residual,
            "l2_relative_error": rel_error,
            "overlap": overlap,
            })


        print(metric)
        result.append(metric)
        i = i + 1

    return result



In [8]:
import random

import numpy as np




def generate_two_points(ranges):
    start = ranges[0]
    end = ranges[1]
    distance = abs(start - end)

    point_1 = np.random.uniform(start, end)
    if point_1 <= (distance/2):
        point_2 = np.random.uniform(point_1, end)
    else:
        point_2 = np.random.uniform(start, point_1)
    points = [round(point_1, 4), round(point_2, 4)]
    return sorted(points)


def generate_box(rangex, rangey):
    return generate_two_points(rangex) + generate_two_points(rangey)


def generate_train_test_boxes(rangex, rangey, overlap=True):
    train_box = generate_box(rangex, rangey)
    if overlap:
        test_box = generate_box(
            [train_box[0], train_box[1]],
             [train_box[2], train_box[3]])
    if not overlap:
        before = bool(random.getrandbits(1))
        if before:
            test_box = generate_box(
                [rangex[0], train_box[0]],
                [rangey[0], train_box[2]])
        else:
            test_box = generate_box(
                [train_box[1], rangex[1]],
                [train_box[2], rangey[1]])

    return train_box, test_box


# p1, p2 = generate_two_points([0, 1])
# print(f"box: {generate_box(range_space, range_time)}")
# print(f"Overlap: {generate_train_test_boxes(range_space, range_time)}")

# print(f"No Overlap: {generate_train_test_boxes(range_space, range_time, overlap=False)}")

# print(f"{p1}, {p2}")

In [10]:
# box: [0.34, 0.8572, 0.1822, 0.8902]
# Overlap: ([0.4022, 0.7992, 0.1667, 0.4093], [0.4703, 0.5556, 0.2372, 0.2502])
# No Overlap: ([0.302, 0.595, 0.6769, 0.7547], [0.0854, 0.271, 0.0915, 0.2096])


import random

def generate_random_condition(rangex, rangey, overlap=True):

    boxes = generate_train_test_boxes(rangex, rangey, overlap)

    return {
        "train_geo_start": boxes[0][0],
        "train_geo_end": boxes[0][1],
        "train_time_start": boxes[0][2],
        "train_time_end": boxes[0][3],

        # test
        "test_geo_start": boxes[0][0],
        "test_geo_end": boxes[0][1],
        "test_time_start": boxes[0][2],
        "test_time_end": boxes[0][3],

    }

def generate_multiple_conditions(n, rangex, rangey, overlap=True):
    return [generate_random_condition(rangex, rangey, overlap) for i in range(n)]


In [11]:
import csv
import time

from google.cloud import storage

STORAGE_CLIENT = storage.Client()
BUCKET = STORAGE_CLIENT.get_bucket('pinns_gabe')


def save_csv(results, overlap):
    file_name = "results_" + str(int(time.time())) + "_.csv"
    with open(file_name, 'w+', newline='') as csvfile:
        fieldnames = ['train_time_start', 'train_time_end','train_geo_start', \
                      'train_geo_end','test_time_start', 'test_time_end', \
                      'test_geo_start', 'test_geo_end', 'mean_residual', \
                      'l2_relative_error', 'overlap']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for item in results:
            writer.writerow(item)

    blob = BUCKET.blob(file_name)
    blob.upload_from_filename(file_name)
    print(f"uploaded {file_name}")

    return file_name

DEBUG:google.auth.transport._http_client:Making request: GET http://169.254.169.254
DEBUG:google.auth.transport._http_client:Making request: GET http://metadata.google.internal/computeMetadata/v1/project/project-id
DEBUG:google.auth.transport._http_client:Making request: GET http://169.254.169.254
DEBUG:google.auth.transport._http_client:Making request: GET http://metadata.google.internal/computeMetadata/v1/project/project-id
DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): metadata.google.internal:80
DEBUG:urllib3.connectionpool:http://metadata.google.internal:80 "GET /computeMetadata/v1/instance/service-accounts/default/?recursive=true HTTP/1.1" 200 187
DEBUG:google.auth.transport.requests:Maki

In [ ]:
number_runs = 50
overlap = True
range_space, range_time = [0,1], [0,1]
conditions = generate_multiple_conditions(number_runs, range_space, range_time, overlap)
result = run_train_eval(conditions, overlap)
file_name = save_csv(result)

Level 2:tensorflow:  _AggregatedGrads 2 x (None, 20) using add_n
Level 1:tensorflow:Gradient for 'gradients_153/gradients_152/dense_102/MatMul_grad/MatMul_grad/MatMul'
Level 1:tensorflow:  in  --> gradients_155/AddN_38:0
Level 1:tensorflow:  out --> gradients_155/gradients_153/gradients_152/dense_102/MatMul_grad/MatMul_grad/MatMul_grad/MatMul:0, gradients_155/gradients_153/gradients_152/dense_102/MatMul_grad/MatMul_grad/MatMul_grad/MatMul_1:0
Level 1:tensorflow:Gradient for 'gradients_152/gradients_150/dense_102/MatMul_grad/MatMul_grad/MatMul'
Level 1:tensorflow:  in  --> gradients_155/gradients_152/gradients_150/dense_102/Tanh_grad/TanhGrad_grad/mul_grad/Mul:0
Level 1:tensorflow:  out --> gradients_155/gradients_152/gradients_150/dense_102/MatMul_grad/MatMul_grad/MatMul_grad/MatMul:0, gradients_155/gradients_152/gradients_150/dense_102/MatMul_grad/MatMul_grad/MatMul_grad/MatMul_1:0
Level 1:tensorflow:Gradient for 'gradients_151/gradients_150/dense_100/Tanh_grad/TanhGrad_grad/mul_2'
Le

'compile' took 5.184594 s

Training model...

Step      Train loss                        Test loss                         Test metric
1         [6.54e-02, 5.70e-03, 6.16e-01]    [6.54e-02, 5.70e-03, 6.16e-01]    []  
1000      [9.52e-03, 1.28e-01, 3.90e-02]    [9.52e-03, 1.28e-01, 3.90e-02]        
2000      [1.03e-02, 1.23e-01, 3.97e-02]    [1.03e-02, 1.23e-01, 3.97e-02]        


INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.170979
  Number of iterations: 2423
  Number of functions evaluations: 2650


2024-05-06 02:48:08,315 tensorflow 0/1 INFO :: Optimization terminated with:
  Message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  Objective function value: 0.170979
  Number of iterations: 2423
  Number of functions evaluations: 2650
2651      [9.64e-03, 1.23e-01, 3.87e-02]    [9.64e-03, 1.23e-01, 3.87e-02]    []  

Best model at step 2651:
  train loss: 1.71e-01
  test loss: 1.71e-01
  test metric: []

'train' took 146.707551 s



DEBUG:distributor:Mesh: []
DEBUG:problems:Adding equation 0
DEBUG:problems:  LHS: dt(u) + -1*0.003183098861837907*dx(dx(u))
DEBUG:problems:  RHS: -1*u*dx(u)
DEBUG:problems:  condition: True
DEBUG:problems:  M: u
DEBUG:problems:  L: -1*0.003183098861837907*dx(dx(u))
DEBUG:problems:  F: -1*u*dx(u)
DEBUG:solvers:Beginning IVP instantiation
INFO:subsystems:Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.7e+01/s


...

DEBUG:solvers:Finished IVP instantiation
DEBUG:transforms:Building FFTW FFT plan for (dtype, gshape, axis) = (<class 'numpy.float64'>, (256,), 0)
DEBUG:transforms:Building FFTW FFT plan for (dtype, gshape, axis) = (<class 'numpy.float64'>, (384,), 0)
INFO:solvers:Simulation stop time reached.


2024-05-06 02:48:11,110 solvers 0/1 INFO :: Simulation stop time reached.


/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
Level 1:tensorflow:Created variable dense_104/kernel:0 with shape (2, 20) and init <tensorflow.python.ops.init_ops.GlorotNormal object at 0x781cd9965ba0>
Level 1:tensorflow:Created variable dense_104/bias:0 with shape (20,) and init <tensorflow.python.ops.init_ops.Zeros object at 0x781c6b651390>
Level 1:tensorflow:Created variable dense_105/kernel:0 with shape (20, 20) and init <tensorflow.python.ops.init_ops.GlorotNormal object at 0x781cd9965ba0>
Level 1:tensorflow:Created variable dense_105/bias:0 with shape (20,) and init <tensorflow.python.ops.init_ops.Zeros object at 0x781c6b651390>
Level 1:tensorflow:Created variable dense_106/kernel:0 with shape (20, 20) and init <tensorflow.python.ops.init_ops.GlorotNormal object at 0x781cd9965ba0>
Level 1:te

{'train_geo_start': 0.1666, 'train_geo_end': 0.5103, 'train_time_start': 0.0407, 'train_time_end': 0.8328, 'test_geo_start': 0.1666, 'test_geo_end': 0.5103, 'test_time_start': 0.0407, 'test_time_end': 0.8328, 'mean_residual': 1.2223635, 'l2_relative_error': 0.5650000304680817, 'overlap': True}
iteration: 27
Compiling model...
Building feed-forward neural network...
'build' took 0.079368 s



Level 1:tensorflow:Gradient for 'dense_107/BiasAdd'
Level 1:tensorflow:  in  --> gradients_156/grad_ys_0:0
Level 1:tensorflow:  out --> gradients_156/grad_ys_0:0, gradients_156/dense_107/BiasAdd_grad/BiasAddGrad:0
Level 1:tensorflow:Gradient for 'dense_107/MatMul'
Level 1:tensorflow:  in  --> gradients_156/grad_ys_0:0
Level 1:tensorflow:  out --> gradients_156/dense_107/MatMul_grad/MatMul:0, gradients_156/dense_107/MatMul_grad/MatMul_1:0
Level 1:tensorflow:Gradient for 'dense_106/Tanh'
Level 1:tensorflow:  in  --> gradients_156/dense_107/MatMul_grad/MatMul:0
Level 1:tensorflow:  out --> gradients_156/dense_106/Tanh_grad/TanhGrad:0
Level 1:tensorflow:Gradient for 'dense_106/BiasAdd'
Level 1:tensorflow:  in  --> gradients_156/dense_106/Tanh_grad/TanhGrad:0
Level 1:tensorflow:  out --> gradients_156/dense_106/Tanh_grad/TanhGrad:0, gradients_156/dense_106/BiasAdd_grad/BiasAddGrad:0
Level 1:tensorflow:Gradient for 'dense_106/MatMul'
Level 1:tensorflow:  in  --> gradients_156/dense_106/Tanh_

In [ ]:
file_name